<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, max_freq=200.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    expected_flashes = int(round(duration * frequency))  # Ensuring approximate expected count
    cycle_length = 1.0 / max(frequency, 0.1)  # Time per cycle, clamped to min 0.1 Hz
    times = []
    time = 0

    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)

        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))

    return times

def regularise_strobe(signal, min_interval=0.025):
    """Regularizes strobe sequence by handling overlaps but maintaining frequency integrity."""
    output_signal = []
    last_flash_end = -np.inf
    for onset, duration in signal:
        if onset >= last_flash_end + min_interval:
            output_signal.append((onset, duration))
            last_flash_end = onset + duration
    return output_signal

def calculate_effective_frequency(signal, duration):
    """Computes the effective frequency based on the number of flashes in the signal."""
    return len(signal) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency, clamping within range."""
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0

    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle, max_freq=max_freq)
        flash_times = regularise_strobe(flash_times)
        Fe = calculate_effective_frequency(flash_times, duration)

        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times

        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1

    print("Warning: Max iterations reached without finding precise Fe match.")
    return F, Fe, flash_times

def generate_session(file_name, stimulation_type):
    """Generates a full session file based on user-defined parameters."""
    num_steps = int(input("Enter number of steps in the session: "))
    durations, frequencies, duty_cycles, brightness, wave_types, led_configs = [], [], [], [], [], []

    for i in range(num_steps):
        print(f"Step {i+1}:")
        durations.append(float(input("Enter duration (s): ")))
        start_Fe = float(input("Enter start effective frequency (Hz): "))
        end_Fe = float(input("Enter end effective frequency (Hz): "))
        frequencies.append((start_Fe, end_Fe))
        start_d = float(input("Enter start duty cycle (0-1): "))
        end_d = float(input("Enter end duty cycle (0-1): "))
        duty_cycles.append((start_d, end_d))
        start_l = int(input("Enter start brightness (0-100): "))
        end_l = int(input("Enter end brightness (0-100): "))
        brightness.append((start_l, end_l))
        wave_types.append(input("Enter wave type (Square/Sine): ").strip())
        led_configs.append([int(input(f"LED Set {j+1} (1=On, 0=Off): ")) for j in range(4)])

    output = [f'TIM"00:00:00:{sum(durations):04.1f}"', f'DUR"{sum(durations):.1f}"']
    min_interval = 0.025
    max_freq = 200.0

    for i in range(num_steps):
        start_Fe, end_Fe = frequencies[i]
        start_d, end_d = int(duty_cycles[i][0] * 100), int(duty_cycles[i][1] * 100)
        start_l, end_l = brightness[i]
        wave_type = 1 if wave_types[i] == "Square" else 2
        led_config = led_configs[i]
        duration = durations[i]

        if stimulation_type == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d, max_freq=max_freq)
        else:
            Fe, flash_times = start_Fe, [(j * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for j in range(int(duration * start_Fe))]
            flash_times = regularise_strobe(flash_times)

        previous_onset = 0
        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration
            step_freq = Fe

            step_string = f'{step_duration:.3f},{wave_type},{start_Fe:.2f},{end_Fe:.2f},{start_d},{end_d},{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{start_l},{end_l},' \
                          f'{wave_type},{start_Fe:.2f},{end_Fe:.2f},{start_d},{end_d},{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{start_l},{end_l},' \
                          f'{wave_type},{start_Fe:.2f},{end_Fe:.2f},{start_d},{end_d},{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{start_l},{end_l},' \
                          f'{wave_type},{start_Fe:.2f},{end_Fe:.2f},{start_d},{end_d},{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{start_l},{end_l}'
            output.append(f'STP"{step_string}"')

    with open(file_name, "w") as f:
        f.write('\n'.join(output))
    print(f"Generated {file_name}")

# User input session generation
type_choice = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
file_name = "strobe_periodic.txt" if type_choice == "Periodic" else "strobe_aperiodic.txt"
generate_session(file_name, type_choice)

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter number of steps in the session: 2
Step 1:
Enter duration (s): 5
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start duty cycle (0-1): .5
Enter end duty cycle (0-1): .5
Enter start brightness (0-100): 50
Enter end brightness (0-100): 50
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
Step 2:
Enter duration (s): 5
Enter start effective frequency (Hz): 15
Enter end effective frequency (Hz): 15
Enter start duty cycle (0-1): .5
Enter end duty cycle (0-1): .5
Enter start brightness (0-100): 50
Enter end brightness (0-100): 50
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
Generated strobe_periodic.txt
